In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import (FloatSlider, FloatLogSlider, 
                        VBox, HBox, Output, Button, 
                        Checkbox, interactive)
import time
from IPython.display import display, clear_output

# Инициализация параметров
params = {
    'Kp': 15.0,
    'Ki': 2.0,
    'Kd': 3.0,
    'setpoint': 50.0,
    'tau': 5.0,
    'sim_time': 30,
    'running': True
}

# Состояние системы
state = {
    'y_current': 25.0,
    'integral_error': 0.0,
    'previous_error': 0.0,
    'time_buffer': np.array([]),
    'temp_buffer': np.array([]),
    'disturbance_buffer': np.array([]),
    'start_time': time.time(),
    'show_disturbance': True
}

# Создание виджетов
kp_slider = FloatLogSlider(value=15, min=np.log10(1), max=np.log10(100), step=0.1, description='Kp:')
ki_slider = FloatLogSlider(value=2, min=np.log10(0.1), max=np.log10(10), step=0.1, description='Ki:')
kd_slider = FloatLogSlider(value=3, min=np.log10(0.1), max=np.log10(10), step=0.1, description='Kd:')
sp_slider = FloatSlider(value=50, min=0, max=100, step=1, description='Setpoint:')

toggle_dist_btn = Button(description='Toggle Disturbance')
stop_btn = Button(description='Stop Simulation')

controls = HBox([
    VBox([kp_slider, ki_slider, kd_slider, sp_slider]),
    VBox([toggle_dist_btn, stop_btn])
])

output = Output()

# Инициализация графика
with output:
    fig, ax = plt.subplots(figsize=(12, 6))
    line_temp, = ax.plot([], [], label='Temperature (°C)', lw=2)
    line_sp = ax.axhline(params['setpoint'], color='r', linestyle='--', label='Setpoint')
    line_dist, = ax.plot([], [], 'g--', alpha=0.5, label='Disturbance')
    ax.set_ylim(0, 100)
    ax.set_xlim(0, params['sim_time'])
    ax.grid(True)
    ax.legend(loc='upper right')
    plt.show()

# Отображаем элементы управления
display(VBox([controls, output]))

# Обработчики событий
def toggle_disturbance(b):
    state['show_disturbance'] = not state['show_disturbance']
    line_dist.set_visible(state['show_disturbance'])
    fig.canvas.draw()

def stop_simulation(b):
    params['running'] = False

toggle_dist_btn.on_click(toggle_disturbance)
stop_btn.on_click(stop_simulation)

def update_system():
    current_time = time.time() - state['start_time']
    dt = 0.1
    
    # Расчет возмущения
    disturbance = 8 * np.sin(0.3 * current_time) + np.random.normal(0, 1)
    
    # PID расчет
    error = sp_slider.value - state['y_current']
    state['integral_error'] += error * dt
    derivative_error = (error - state['previous_error']) / dt
    
    u = (kp_slider.value * error + 
         ki_slider.value * state['integral_error'] + 
         kd_slider.value * derivative_error)
    
    # Модель системы
    state['y_current'] += (-state['y_current'] + u + disturbance) / params['tau'] * dt
    state['previous_error'] = error
    
    # Обновление данных
    state['time_buffer'] = np.append(state['time_buffer'], current_time)
    state['temp_buffer'] = np.append(state['temp_buffer'], state['y_current'])
    state['disturbance_buffer'] = np.append(state['disturbance_buffer'], disturbance)
    
    # Обрезка старых данных
    keep_mask = state['time_buffer'] > (current_time - params['sim_time'])
    state['time_buffer'] = state['time_buffer'][keep_mask]
    state['temp_buffer'] = state['temp_buffer'][keep_mask]
    state['disturbance_buffer'] = state['disturbance_buffer'][keep_mask]

def update_plot():
    line_temp.set_data(state['time_buffer'], state['temp_buffer'])
    line_sp.set_ydata([sp_slider.value])
    line_dist.set_data(state['time_buffer'], state['disturbance_buffer'])
    
    ax.set_xlim(max(0, state['time_buffer'][-1] - params['sim_time'] if len(state['time_buffer']) > 0 else 0,
               max(params['sim_time'], state['time_buffer'][-1] if len(state['time_buffer']) > 0 else params['sim_time'])))
    
    fig.canvas.draw()

try:
    while params['running']:
        with output:
            update_system()
            update_plot()
            clear_output(wait=True)
            display(fig)
        time.sleep(0.05)
except KeyboardInterrupt:
    params['running'] = False
finally:
    print("Simulation stopped")
